# Binary Quantization with Qdrant & OpenAI Embedding

---
In the world of large-scale data retrieval and processing, efficiency is crucial. With the exponential growth of data, the ability to retrieve information quickly and accurately can significantly affect system performance. This blog post explores a technique known as binary quantization applied to OpenAI embeddings, demonstrating how it can enhance **retrieval latency by 20x** or more.

## What Are OpenAI Embeddings?
OpenAI embeddings are numerical representations of textual information. They transform text into a vector space where semantically similar texts are mapped close together. This mathematical representation enables computers to understand and process human language more effectively.

## Binary Quantization
Binary quantization is a method which converts continuous numerical values into binary values (0 or 1). It simplifies the data structure, allowing faster computations. Here's a brief overview of the binary quantization process applied to OpenAI embeddings:

1. **Load Embeddings**: OpenAI embeddings are loaded from parquet files.
2. **Binary Transformation**: The continuous valued vectors are converted into binary form. Here, values greater than 0 are set to 1, and others remain 0.
3. **Comparison & Retrieval**: Binary vectors are used for comparison using logical XOR operations and other efficient algorithms.

Binary Quantization is a promising approach to improve retrieval speeds and reduce memory footprint of vector search engines. In this notebook we will show how to use Qdrant to perform binary quantization of vectors and perform fast similarity search on the resulting index.

## Table of Contents
1. Imports
2. Download and Slice Dataset
3. Create Qdrant Collection
4. Indexing
5. Search

## 1. Imports

In [1]:
!pip install qdrant-client pandas dataset --quiet --upgrade

In [2]:
import pandas as pd
from qdrant_client import QdrantClient, models

## 2. Download and Slice Dataset

We will be using the [dbpedia-entities](https://huggingface.co/datasets/Qdrant/dbpedia-entities-openai3-text-embedding-3-small-1536-100K) dataset from the [HuggingFace Datasets](https://huggingface.co/datasets) library. This contains 100K vectors of 1536 dimensions each

In [4]:
import datasets

dataset = datasets.load_dataset(
    "Qdrant/dbpedia-entities-openai3-text-embedding-3-small-1536-100K", split="train"
)

In [5]:
len(dataset)
# dataset[0]

100000

In [8]:
client = QdrantClient(
    ":memory:",
    timeout=600,
    prefer_grpc=True,
)

collection_name = "binary-quantization"
client.recreate_collection(
    collection_name=f"{collection_name}",
    vectors_config=models.VectorParams(
        size=1536,
        distance=models.Distance.DOT,
        on_disk=True,
    ),
    quantization_config=models.BinaryQuantization(
        binary=models.BinaryQuantizationConfig(always_ram=True),
    ),
)

True

In [9]:
import os


bs = 1000
for i in range(0, len(dataset), bs):
    client.upload_collection(
        collection_name=collection_name,
        ids=range(i, i + bs),
        vectors=dataset[i : i + bs]["openai"],
        payload=[{"text": x} for x in dataset[i : i + bs]["text"]],
        parallel=max(1, (os.cpu_count() // 2)),
    )

In [10]:
collection_info = client.get_collection(collection_name=f"{collection_name}")
collection_info.dict()

{'status': <CollectionStatus.GREEN: 'green'>,
 'optimizer_status': <OptimizersStatusOneOf.OK: 'ok'>,
 'vectors_count': 100000,
 'indexed_vectors_count': 0,
 'points_count': 100000,
 'segments_count': 1,
 'config': {'params': {'vectors': {'size': 1536,
    'distance': <Distance.DOT: 'Dot'>,
    'hnsw_config': None,
    'quantization_config': None,
    'on_disk': True},
   'shard_number': None,
   'sharding_method': None,
   'replication_factor': None,
   'write_consistency_factor': None,
   'read_fan_out_factor': None,
   'on_disk_payload': None,
   'sparse_vectors': None},
  'hnsw_config': {'m': 16,
   'ef_construct': 100,
   'full_scan_threshold': 10000,
   'max_indexing_threads': 0,
   'on_disk': None,
   'payload_m': None},
  'optimizer_config': {'deleted_threshold': 0.2,
   'vacuum_min_vector_number': 1000,
   'default_segment_number': 0,
   'max_segment_size': None,
   'memmap_threshold': None,
   'indexing_threshold': 20000,
   'flush_interval_sec': 5,
   'max_optimization_thread

## Oversampling vs Recall

### Preparing a query dataset

For the purpose of this illustration, we'll take a few vectors which we know are already in the index and query them. We should get the same vectors back as results from the Qdrant index. 

In [11]:
import random
from random import randint

random.seed(37)

query_indices = [randint(0, len(dataset)) for _ in range(100)]
query_dataset = dataset[query_indices]
query_indices

[89391,
 79659,
 12006,
 80978,
 87219,
 97885,
 83155,
 67504,
 4645,
 82711,
 48395,
 57375,
 69208,
 14136,
 89515,
 59880,
 78730,
 36952,
 49620,
 96486,
 55473,
 58179,
 18926,
 6489,
 11931,
 54146,
 9850,
 71259,
 37825,
 47331,
 84964,
 92399,
 56669,
 77042,
 73744,
 47993,
 83780,
 92429,
 75114,
 4463,
 69030,
 81185,
 27950,
 66217,
 54652,
 8260,
 1151,
 993,
 85954,
 66863,
 47303,
 8992,
 92688,
 76030,
 29472,
 3077,
 42454,
 46120,
 69140,
 20877,
 2844,
 95423,
 1770,
 28568,
 96448,
 94227,
 40837,
 91684,
 29785,
 66936,
 85121,
 39546,
 81910,
 5514,
 37068,
 35731,
 93990,
 26685,
 63076,
 18762,
 27922,
 34916,
 80976,
 83189,
 6328,
 57508,
 58860,
 13758,
 72976,
 85030,
 332,
 34963,
 85009,
 31344,
 11560,
 58108,
 85163,
 17064,
 44712,
 45962]

In [12]:
## Add Gaussian noise to any vector
import numpy as np

np.random.seed(37)


def add_noise(vector, noise=0.05):
    return vector + noise * np.random.randn(*vector.shape)

In [13]:
import time


def correct(results, text):
    result_texts = [x.payload["text"] for x in results]
    return text in result_texts


def count_correct(query_dataset, limit=1, oversampling=1, rescore=False):
    correct_results = 0
    for qv, text in zip(query_dataset["openai"], query_dataset["text"]):
        results = client.search(
            collection_name=collection_name,
            query_vector=add_noise(np.array(qv)),
            limit=limit,
            search_params=models.SearchParams(
                quantization=models.QuantizationSearchParams(
                    ignore=False,
                    rescore=rescore,
                    oversampling=oversampling,
                )
            ),
        )
        correct_results += correct(results, text)
    return correct_results


limit_grid = [1, 3, 5, 10, 20, 50]
# limit_grid = [1, 3, 5]
oversampling_grid = [1.0, 1.5, 2.0, 3.0, 5.0]
# oversampling_grid = [1.0, 1.5, 2.0]
rescore_grid = [False, True]
results = []
for limit in limit_grid:
    for oversampling in oversampling_grid:
        for rescore in rescore_grid:
            # print(f"limit={limit}, oversampling={oversampling}, rescore={rescore}")
            start = time.time()
            correct_results = count_correct(
                query_dataset, limit=limit, oversampling=oversampling, rescore=rescore
            )
            end = time.time()
            results.append(
                {
                    "limit": limit,
                    "oversampling": oversampling,
                    "rescore": rescore,
                    "correct": correct_results,
                    "total queries": len(query_dataset["text"]),
                    "time": end - start,
                }
            )

results_df = pd.DataFrame(results)
results_df

In [ ]:
df = results_df.copy()
df["candidates"] = df["oversampling"] * df["limit"]
df[["candidates", "rescore", "time"]]
# df.to_csv("candidates-rescore-time.csv", index=False)